In [1]:
# On importe les librairies
import numpy as np # calcul matriciel
import pandas as pd # structure des donnees
import matplotlib.pyplot as plt # graphique

# Chemin vers les jeux de donnees en format csv
pathTest = "./test3.csv"
pathTrain = "./train3.csv"

# On importe les jeux de donnees
df_test = pd.read_csv(pathTest, sep=',', index_col=0)
df_train = pd.read_csv(pathTrain, sep=',', index_col=0)


# --- Variables Explicatives ---

#    Name : Nom du jeu
#    Platform : Console sur laquelle le jeu fonctionne
#    Year_of_Release : Année de sortie du jeu
#    Genre
#    Publisher
#    JP_Sales : Nombre de ventes du jeu au Japon en millions d’unités
#    Other_Sales : Nombre de ventes du jeu ailleurs dans le monde : Afrique, Asie sans le Japon, Europe sans l’Union Européenne et Amérique du Sud en millions d’unités
#    Critic_Score : Score donné par Metacritic
#    Critic_Count : Nombre de critiques prises en compte pour estimer le Critic_score
#    User_Score : Score donné par les usagers de Metacritic
#    User_Count : Nombre d’usagers considérés pour estimer le User_Score
#    Developer : Compagnie créatrice du jeu
#    Rating : Classement ESRB (Entertainment Software Rating Board) ie à qui s’addresse le jeu (tout public, majeur, adolescents, etc) 


# --- Variables d'Interet ---

#    NA_sales : Nombre de ventes du jeu en Amérique du Nord en millions d’unités
#    Global_Sales : Nombre de ventes total du jeu en millions d’unités

In [2]:
"""

On definit ici la fonction qui calcule les coefficients de notre regression lineaire

Arguments :

    y : Variable d'interet (dimension : n x 1)
    x : matrice contenant nos variables explicatives (dimension : n x p)
    
    ou,
        n : nombre d'observation
        p : nombre de variable explicative
        
        

Retourne : 

    B = coefficiants de regression

    Pour estimer la valeur de y_new avec de nouvelles observations (i.e. x_new) on a,
        y_new = np.dot(x_new,B)

"""

def linear_regression(y,x):
    
    # On calcule la matrice de variance-covariance
    C = np.linalg.inv(np.dot(x.T,x))
    
    # On calcule les coefficients de regression
    B = np.dot(np.dot(C,x.T),y)
    
    return B

In [3]:
"""
--- Variable Qualitative -> Table Binaire --- 

Afin de traiter nos variables qualitatives, nous avons une fonction qui la tranforme en table binaire 
avec un one bit encoder. 

    Disons que la variable explicative peut prendre 4 valeurs {Bleu, Vert, Jaune, Rouge},
        
          ID        X1
        ----------------
          1        Bleu
          2        Bleu
          3        Vert
          4        Jaune
          5        Vert
          6        Rouge          
         ...
    
    Suite a un encodage en 1 bit notre variable explicative X1 devient,
    
          ID        Bleu        Vert        Jaune        Rouge
        -------------------------------------------------------
          1          1           0            0            0
          2          1           0            0            0
          3          0           1            0            0
          4          0           0            1            0
          5          0           1            0            0
          6          0           0            0            1
                                ...
                                

Nous nous basons sur la fonction get_dummies() de la librairie Panda pour faire ceci.


Arguments,

    trainSet : le jeu de donnees de training
    testSet : le jeu de donnees de test
    varName : la variable qualitative que nous souhaitons transformer en tableau binaire
    
    
Retourne, 
    trainSet_new : le jeu de donnees de training ou la variable qualitative a ete remplacee par un tableau binaire
    testSet_new : le jeu de donnees de test ou la variable qualitative a ete remplacee par un tableau binaire

"""

def qualiToBinary(trainSet, testSet, varName):
    
    # Genere la table binaire de la variable qualitative
    trainVar_binary = pd.get_dummies(trainSet[varName],prefix=varName)
    testVar_binary = pd.get_dummies(testSet[varName],prefix=varName)
    
    # On convertit nos tableaux binaires en Dataframe
    trainVar_binary = pd.DataFrame(trainVar_binary)
    testVar_binary = pd.DataFrame(testVar_binary)
    
    # On enleve la variable qualitative de nos jeux de donnees
    trainSet_new = trainSet.drop([varName], axis=1)
    testSet_new = testSet.drop([varName], axis=1)   

    # On ajoute les nouvelles colonnes a nos jeux de donnees
    trainSet_new = pd.concat([trainSet_new,trainVar_binary],axis=1)
    testSet_new = pd.concat([testSet_new,testVar_binary],axis=1)
    
    # On identifie les colonnes manquantes dans le jeu de donnees test
    missing_categories = set(trainSet_new) - set(testSet_new)

    # On ajoute ces colonnes manquantes avec une valeur par defaut de 0
    for c in missing_categories:
        testSet_new[c] = 0

    # On identifie les colonnes manquantes dans le jeu de donnees training
    missing_categories = set(testSet_new) - set(trainSet_new)

    # On ajoute ces colonnes manquantes avec une valeur par defaut de 0
    for c in missing_categories:
        trainSet_new[c] = 0


    # On aligne nos jeux de donnees pour que les memes colonnes soient aux memes endroits
    trainSet_new, testSet_new = trainSet_new.align(testSet_new, axis=1)
    
    # On retourne les jeux de donnees
    return trainSet_new, testSet_new

In [4]:
"""

L'algorithme R2_prev implemente une validation croise sur l'ensemble du jeu de donnees fournis

Arguments:
    Y : La variable d'interet
    W : Notre estimation de la variable d'interet
    X : Nos variables explicatives
    
    
Retourne:
    R2_prev : Une mesure bornee entre [-inf, 1] qui represente le pouvoir predictif de nos estimations

"""

def R2_prev(Y,W,X):
    
    # The variance-covariance
    C = np.linalg.inv(np.dot(X.T,X))
    
    # Hat Matrix (X*B = H*Y)
    H = np.dot(np.dot(X,C),X.T)
    
    # We are only interested in the (i,i) values
    H = np.diagonal(H)
    
    # Mean of Y
    y_S = np.sum(Y)/float(len(Y))
    
    num = 0.0
    for i in range(len(Y)):
        num += ((Y[i] - W[i])/float(1 - H[i]))**2

    denom = 0.0
    for i in range(len(Y)):
        denom += (Y[i] - y_S)**2
    
    R2_prev = (1 - num/denom)
    
    return R2_prev
    

In [5]:
"""
Fonction qui affiche la quantite de NaN par colonne

Argument:
    dataset : Le jeu de donnees sur lequel nous appliquons la fonction

"""

def printNaN(dataset):
    for col in dataset:
        if(dataset[col].isna().sum() > 0):
            print('%d \t %s' % (dataset[col].isna().sum(),col))

In [6]:
"""
Fonction permettant de cree une matrice contenant chaque combinaison binaire de largeur n

Argument:
    n : nombre de bit
    
Retourne:
    matrix : matrice de dimension (2^n x n) contenant toutes les combinaisons

"""

def allBinaryCombinations(n):
    
    combinations = np.meshgrid(*[[0,1]]*n)
    
    matrix = np.array(combinations).reshape((n,-1)).T
    
    return matrix

In [7]:
"""
Fonction qui affiche la valeur moyenne de la variable d'interet en fonction des 
valeurs d'une variable explicative qualitative.

Arguments:
    dataset : Le jeu de donnees sur lequel nous appliquons la fonction
    Y : la variable d'interet
    colName : Le nom de la variable qualitative que nous souhaitons etudier

"""

def clusterVar(dataset,Y,colsName):
    
    # On selectionne seulement les colonnes qui debute par 'colsName'
    reducedSet = dataset.loc[:, dataset.columns.str.startswith(colsName)]
    
    # On calcule le nombre de colonne
    nbrOfCols = len(reducedSet.T)
    
    # On cree une liste d'index de 0 au nombre de colonne
    rangeIndex = np.array(range(nbrOfCols))
    
    sumWithX = np.zeros(nbrOfCols)
    varWithX = np.zeros(nbrOfCols)
    valueCount = np.zeros(nbrOfCols)
    names = np.array(['' for _ in range(nbrOfCols)], dtype=object)
    index = 0
    for col in reducedSet:
        withX = reducedSet[col]*Y
        sumWithX[index] = np.sum(withX)/(withX.astype(bool).sum(axis=0))
        varWithX[index] = np.var(withX)
        valueCount[index] = reducedSet[col].astype(bool).sum(axis=0)
        names[index] += col
        index += 1
    
    # On cree un tableau indexe de nos categories, leur quantite et leur valeur moyenne de la variable d'interet
    aveY_table = np.array(([rangeIndex,sumWithX,varWithX,valueCount,names])).T
    
    # On ordonne en ordre decroissant de contribution le tableau
    aveY_table_ordered = aveY_table[(-aveY_table[:,1]).argsort()]
    
    print('Index \t Moy \t Var \t Nbr \t Name')    
    print('------------------------------------------')
    for i in range(nbrOfCols):
        print('%d. \t %.3f \t %.3f \t %d \t %s ' % (aveY_table_ordered[i][0],aveY_table_ordered[i][1],aveY_table_ordered[i][2],aveY_table_ordered[i][3],aveY_table_ordered[i][4]))

In [8]:
# Variable Qualitative #1 : Platform
print("Nbr. of categories (train) : %d" %(df_train.Platform.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Platform.nunique()))

df_train, df_test = qualiToBinary(df_train, df_test, 'Platform')

Nbr. of categories (train) : 30
Nbr. of categories (test) : 27


In [9]:
# Variable Qualitative #2 : Genre
print("Nbr. of categories (train) : %d" %(df_train.Genre.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Genre.nunique()))

df_train, df_test = qualiToBinary(df_train, df_test,'Genre')

Nbr. of categories (train) : 12
Nbr. of categories (test) : 12


In [10]:
# Variable Qualitative #3 : Publisher
print("Nbr. of categories (train) : %d" %(df_train.Publisher.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Publisher.nunique()))

#df_train, df_test = qualiToBinary(df_train, df_test, 'Publisher')
df_test = df_test.drop(['Publisher'],axis=1)
df_train = df_train.drop(['Publisher'],axis=1)

Nbr. of categories (train) : 552
Nbr. of categories (test) : 280


In [11]:
# Variable Qualitative #4 : Developer
print("Nbr. of categories (train) : %d" %(df_train.Developer.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Developer.nunique()))


# Bon ici on a un probleme, si on ajoute les colonnes binaires generees par cette variable on ajoute 100mb
# a notre csv. Ceci rend peu pratique le prototypage, pour l'instant on le laisse tomber. On essayera de ce
# donner une raison rationnelle de le domper plus tard dans l'analyse. Hypothese, beaucoup de colinearite avec 
# le Publisher.

#df_train, df_test = qualiToBinary(df_train, df_test,'Developer')
df_test = df_test.drop(['Developer'],axis=1)
df_train = df_train.drop(['Developer'],axis=1)

Nbr. of categories (train) : 1593
Nbr. of categories (test) : 677


In [12]:
# Variable Qualitative #5 : Rating
print("Nbr. of categories (train) : %d" %(df_train.Rating.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Rating.nunique()))

df_train, df_test = qualiToBinary(df_train, df_test,'Rating')

Nbr. of categories (train) : 8
Nbr. of categories (test) : 5


In [13]:
df_train_clean = df_train.fillna(df_train.median()).copy()

In [14]:
def createX_Global(testSet):
    
    # Platform
    X8_1 = testSet.Platform_NES.values
    X8_2 = testSet.Platform_GB.values
    X8_3 = testSet.Platform_GEN.values
    X8_4 = testSet.Platform_SNES.values
    X8_5 = testSet.Platform_X360.values
    X8_6 = testSet.Platform_PS4.values
    X8_7 = testSet.Platform_2600.values
    X8_8 = testSet.Platform_Wii.values
    X8_9 = testSet.Platform_PS3.values
    X8_10 = testSet.Platform_N64.values
    X8_11 = testSet.Platform_XOne.values
    X8_12 = testSet.Platform_WiiU.values
    X8_13 = testSet.Platform_PS.values
    X8_14 = testSet.Platform_PS2.values
    X8_15 = testSet.Platform_3DS.values
    X8_16 = testSet.Platform_GG.values # ?
    X8_17 = testSet.Platform_GBA.values
    X8_18 = testSet.Platform_GC.values
    X8_19 = testSet.Platform_DS.values
    X8_20 = testSet.Platform_XB.values
    X8_21 = testSet.Platform_DC.values
    X8_22 = testSet.Platform_SCD.values
    X8_23 = testSet.Platform_PC.values
    X8_24 = testSet.Platform_PSP.values
    X8_25 = testSet.Platform_SAT.values
    X8_26 = testSet.Platform_WS.values
    X8_27 = testSet.Platform_PSV.values
    X8_28 = testSet.Platform_NG.values
    X8_29 = testSet.Platform_TG16.values
    X8_30 = testSet.Platform_3DO.values
    X8_31 = testSet.Platform_PCFX.values

    # Genre    
    X9_1 = testSet.Genre_Platform.values
    X9_2 = testSet.Genre_Shooter.values
    X9_3 = testSet.Genre_Racing.values
    X9_4 = testSet.Genre_Sports.values
    X9_5 = testSet['Genre_Role-Playing'].values
    X9_6 = testSet.Genre_Action.values
    X9_7 = testSet.Genre_Fighting.values
    X9_8 = testSet.Genre_Misc.values
    X9_9 = testSet.Genre_Puzzle.values
    X9_10 = testSet.Genre_Simulation.values
    X9_11 = testSet.Genre_Strategy.values
    X9_12 = testSet.Genre_Adventure.values
    
    X1 = testSet.JP_Sales.values
    X2 = testSet.Critic_Count.values
    X3 = testSet.Critic_Score.values
    X4 = testSet.Year_of_Release.values
    X5 = testSet.Other_Sales.values
    X6 = testSet.User_Score.values
    X7 = testSet.User_Count.values
    
    # Par tattonnement on arrive a
    X = np.array([np.ones(len(testSet.JP_Sales)),
                  X1,X1**2,X1*X5,X1*X4,X5*X4,X5**2,X4,(X3**12)*X2,X2,X3,X6,X7,X7**2,
                  X8_3,X8_4,X8_5,X8_7+X8_8+X8_10+X8_11,X8_17+X8_18+X8_19,X8_20,X8_25,
                  X9_1,X9_2,X9_3+X9_4,X9_5,X9_11]).T
    
    return X

In [15]:
Y1 = df_train_clean.Global_Sales.values
X1 = createX_Global(df_train_clean)
B1 = linear_regression(Y1,X1)

# Clip to min 0
global_data = np.dot(X1,B1)
global_data = global_data.clip(min=0)

print("R2_prev (Global) : %.4f" % (R2_prev(Y1,global_data,X1)))
# Max 0.7850

R2_prev (Global) : 0.7850


In [16]:
def createX_NA(testSet):
    
    # Platform
    X8_1 = testSet.Platform_NES.values
    X8_2 = testSet.Platform_GB.values
    X8_3 = testSet.Platform_GEN.values
    X8_4 = testSet.Platform_2600.values
    X8_5 = testSet.Platform_X360.values
    X8_6 = testSet.Platform_N64.values
    X8_7 = testSet.Platform_Wii.values
    X8_8 = testSet.Platform_GG.values
    X8_9 = testSet.Platform_XOne.values
    X8_10 = testSet.Platform_PS3.values
    X8_11 = testSet.Platform_SNES.values
    X8_12 = testSet.Platform_WiiU.values
    X8_13 = testSet.Platform_PS.values
    X8_14 = testSet.Platform_PS2.values
    X8_15 = testSet.Platform_PS4.values
    X8_16 = testSet.Platform_GC.values # ?
    X8_17 = testSet.Platform_GBA.values
    X8_18 = testSet.Platform_XB.values
    X8_19 = testSet.Platform_3DS.values
    X8_20 = testSet.Platform_SCD.values
    X8_21 = testSet.Platform_DS.values
    X8_22 = testSet.Platform_DC.values
    X8_23 = testSet.Platform_PC.values
    X8_24 = testSet.Platform_PSP.values
    X8_25 = testSet.Platform_PSV.values
    X8_26 = testSet.Platform_SAT.values
    X8_27 = testSet.Platform_PCFX.values
    X8_28 = testSet.Platform_WS.values
    X8_29 = testSet.Platform_3DO.values
    X8_30 = testSet.Platform_TG16.values
    X8_31 = testSet.Platform_NG.values
    

    # Genre    
    X9_1 = testSet.Genre_Platform.values
    X9_2 = testSet.Genre_Shooter.values
    X9_3 = testSet.Genre_Sports.values
    X9_4 = testSet.Genre_Racing.values
    X9_5 = testSet.Genre_Action.values
    X9_6 = testSet.Genre_Fighting.values
    X9_7 = testSet.Genre_Misc.values
    X9_8 = testSet.Genre_Puzzle.values
    X9_9 = testSet['Genre_Role-Playing'].values
    X9_10 = testSet.Genre_Simulation.values
    X9_11 = testSet.Genre_Strategy.values
    X9_12 = testSet.Genre_Adventure.values
    
    X1 = testSet.JP_Sales.values
    X2 = testSet.Critic_Count.values
    X3 = testSet.Critic_Score.values
    X4 = testSet.Year_of_Release.values
    X5 = testSet.Other_Sales.values
    X6 = testSet.User_Score.values
    X7 = testSet.User_Count.values

    X = np.array([np.ones(len(testSet.JP_Sales)),
                  #X1,X1**2,X1*X5,X1*X4,X5*X4,X5**2,X4,(X3**12)*X2,X2,X3,X6,X7,X7**2,
                  X1**2,X1,X1*X5,X5,X1**2*X4**2,X4,X5**2,(X3**9)*X2,
                  X8_4+X8_5+X8_6+X8_7+X8_9,X8_11,X8_16+X8_18,X8_26,
                  X9_1,X9_9]).T
    
    return X

In [17]:
Y2 = df_train_clean.NA_Sales.values
X2 = createX_NA(df_train_clean)
B2 = linear_regression(Y2,X2)
  
# Clip to min 0
na_data = np.dot(X2,B2)
na_data = na_data.clip(min=0)

print("NA R2_prev : %.3f" % (R2_prev(Y2,na_data,X2)))

# Max 0.592

NA R2_prev : 0.592


In [18]:
#clusterVar(df_train_clean,df_train_clean.NA_Sales.values,'Genre')

In [19]:
df_test_clean = df_test.fillna(df_test.median()).copy()

X1_test = createX_Global(df_test_clean)
X2_test = createX_NA(df_test_clean)

test_GlobalData = np.dot(X1_test,B1)
test_NAData = np.dot(X2_test,B2)

# Clip to min 0
test_GlobalData = test_GlobalData.clip(min=0)  

# Clip to min 0
test_NAData = test_NAData.clip(min=0)  

In [20]:
df_test_estimated = pd.DataFrame([df_test.NA_Sales,df_test.Global_Sales]).copy().T
df_test_estimated['NA_Sales'] = test_NAData
df_test_estimated['Global_Sales'] = test_GlobalData
df_test_estimated.to_csv("test_final.csv")